In [1]:
import pandas
import ee  # Google Earth Engine API
import folium
import numpy as np

In [2]:
ee.Initialize()

In [3]:

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Source: 
    https://developers.google.com/earth-engine/python_install-colab.html#interactive_map
    """
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data © Google Earth Engine",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

def add_vector_layer(self, ee_vector_object, vis_params, name):
    map_id_dict = ee.Image(ee_vector_object).getMapId(vis_params)
    print(map_id_dict)
    print(map_id_dict['tile_fetcher'].url_format)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data © Google Earth Engine",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)
    
def display_map(dataset):
    """
    Display an interactive map using folium as suggested here:
    https://developers.google.com/earth-engine/python_install-colab.html#interactive_map
    """
    
    # If the dataset is in vector format (as a FeatureCollection object), it needs to be
    # converted to raster format (an Image object) to display with folium
    if isinstance(dataset, ee.FeatureCollection):
        print("yes")
        dataset = dataset.reduceToImage(properties=['us_l3code'], reducer=ee.Reducer.first())
        
    # Add EE drawing method to folium.
    folium.Map.add_ee_layer = add_ee_layer
    folium.Map.add_vector_layer = add_vector_layer

    # Set visualization parameters.
    vis_params = {
      'min': 1,
      'max': 120,
      'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

    # Create a folium map object.
    my_map = folium.Map(location=[40, -100], zoom_start=4, height=500)

    # Add the dataset to the map object.
    #my_map.add_ee_layer(dataset.updateMask(dataset.gt(0)), vis_params, 'DEM')
    sample_location = ee.Geometry.Point(34.083974, -118.567312).buffer(100000).coordinates()
    print(sample_location.getInfo())
    print(np.array((ee.Array(sample_location.getInfo())).getInfo()))
    my_map.add_vector_layer(sample_location, vis_params, "sample")

    # Add a layer control panel to the map.
    my_map.add_child(folium.LayerControl())

    # Display the map.
    display(my_map)


In [4]:
def to_number(key: str):
    """
    Return a lambda function that converts a feature's property value from a string to a number.
    To be used with map().
    
    param key: the name of the feature property to convert
    """
    return lambda feature: feature.set({key: ee.Number.parse(feature.get(key))})

dataset = ee.FeatureCollection('EPA/Ecoregions/2013/L3')
dataset = dataset.map(to_number(key='us_l3code'))
display_map(dataset)


yes
[[[-145.91602600000002, -60.532801404851874], [-146.4275146149715, -60.56763424976949], [-146.90143308393678, -60.66957567536129], [-147.3023032631838, -60.83109734486689], [-147.59889311177753, -61.04013202853984], [-147.76644921747553, -61.28080288397563], [-147.7889330214365, -61.534453782356266], [-147.66105766794163, -61.78096797881946], [-147.3897835830262, -62.00032797557517], [-146.99483535099793, -62.17431764453806], [-146.5078230652773, -62.28820780892083], [-145.96974576199818, -62.33221847597284], [-145.42701315078838, -62.30254040220077], [-144.92653547644957, -62.201744153380645], [-144.5107243628353, -62.03850336911378], [-144.21327666134158, -61.826682091182896], [-144.05636372056588, -61.583942307919465], [-144.04944402434612, -61.33008481648424], [-144.18953610804576, -61.085336003309024], [-144.4625583474992, -60.86875046513579], [-144.84529074932252, -60.69683997479598], [-145.30759257758942, -60.58248486061727], [-145.81464508661378, -60.53414577602442], [-145.

EEException: Image.visualize, argument 'image': Invalid type. Expected: Image<unknown bands>. Actual: List<List<List<Float>>>.